In [ ]:
import os 
import sys
import pandas as pd

sys.path.append(os.path.abspath(os.path.join('..', '..')))

from services.database.database_service import Environment

In [ ]:
conn = Environment().get_instance()
db = conn.get_db()

likes_collection = db.get_collection("LikesCollection")

In [ ]:
df = pd.DataFrame.from_records(likes_collection.find({'is_liked': True}))
df

In [ ]:
pd.DataFrame.from_records(likes_collection.find())

see: https://www.mongodb.com/docs/manual/reference/operator/aggregation/count/


In [ ]:


list(likes_collection.aggregate([
    {'$match': {'is_liked': True}},
    {'$group': {'_id': '661f2d5e30ee4c696a1d15fd', 'detail': { '$first': '$$ROOT' },'count': {'$sum': 1}}},
    {'$replaceRoot': { 'newRoot': { '$mergeObjects': [{ 'count': '$count' }, '$detail'] }}},
    ]))

In [ ]:
list(likes_collection.aggregate([
    {'$match': {'is_liked': True}},
    {'$group': {'_id': {'user_id': '$$ROOT.user_id'}, 'like_count': {'$sum': 1}} },
    ]))

In [ ]:
# { $count: "passing_scores" }
result = likes_collection.aggregate([
    {'$match': {'is_liked': True, 'user_id': '661f2d5e30ee4c696a1d15fd'}},
    {'$count': "liked_count" },
    ])
list(result)

In [ ]:
list(likes_collection.aggregate([
    {'$match':{'is_liked': True, 'user_id':'661f2d5e30ee4c696a1d15fd'}},
    {"$group" :{"_id":"$name", "data" : {"$first":"$$ROOT"}, 'count': {'$sum': 1}}},
    {"$project":{ "happen_id":"$data.happen_id", "user_id" : "$data.user_id", "count" : "$count", "_id" : "$data._id"}}
    ]))

In [ ]:
list(likes_collection.aggregate([
    {'$match':{'$and': [{'is_liked': True, 'user_id':'661f2d5e30ee4c696a1d15fd'}]}},
    
    {"$group" :{"_id":"$name", "data" : {"$first":"$$ROOT"}, 'count': {'$sum': 1}}},
    {"$project":{ "happen_id":"$data.happen_id", "user_id" : "$data.user_id", "like_count" : "$count", "_id" : "$data._id"}},

    ]))

see: https://www.mongodb.com/community/forums/t/how-to-use-multiple-conditions-in-match-and-sum-in-group/191477/3


In [ ]:
list(likes_collection.aggregate([
    {'$match': { '$and': [ { 'createdAt': {'$gte': ('2022-09-15'), '$lt':('2022-09-16') }} ]}},
    {'$group':{ '_id': {'createdAt': {'$last': "$createdAt"}}, 'totalAmount': { '$sum': "$postBalance" }}}
    ]))

In [ ]:
list(likes_collection.aggregate([
  {
    '$lookup': {
      'from': "LikesCollection",
      'let': {
        'liked': "$is_liked",
        'disliked': "$is_disliked"
      },
      'pipeline': [
        {
          '$match': {
            '$expr': {
              '$and': [
                {
                  'is_liked': True, 'user_id':'661f2d5e30ee4c696a1d15fd'
                },
                # {
                #   '$eq': [ "$disliked", False]
                # }
              ]
            }
          }
        },
        {
            "$group" : {
                "_id": "$name", 
                "data": { "$first":"$$ROOT" }, 
                'count': {'$sum': 1}}
        },
      ],
      'as': "joined_results"
    }
  },
  {
    '$unwind': { 'path': "$joined_results" }
  }
]))

In [ ]:
pipeline = [
  # first, group ids of buyers and sellers
  {
    '$group': {
      '_id': None,
      'userId': {
        '$addToSet': '$user_id',
      },
      'happenId': {
        '$addToSet': '$happen_id',
      },
    },
  },
  {
      '$group': {
          '_id': None,
          'liked': {
            '$addToSet': '$userId'
          },
        'disliked': {
            '$addToSet': '$is_disliked'
        }
      }
  },
  # then, concat grouped ids into single array 
  # to fetch users with one single $lookup
  # ids in the array will not be unique, 
  # but that does not matter for $lookup
 
  # PS: you can $unwind 'userIsds' 
  # and then $group them like we did with 'like_id' above,
  # if unique values in 'usersIds' bother you :)
  {
    '$project': {
      'userId': {
        '$concatArrays': ['$userId', '$happenId'],
      },
      'liked_count': {
          '$concatArrays': ['$liked']
      },
      'disliked_count': {
          '$concatArrays': ['$disliked']
        }
    },
  },
  # use $lookup to join users by ids from accumulated array
  {
    '$lookup': {
      'from': 'LikesCollection',
      'localField': 'liked_count',
      'foreignField': '_id',
      'as': 'likes',
    },
  },
  # unwind 'users' to be bring out each user to upper level
  # {
  #   '$unwind': '$users',
  # },
  # # make each user a root object in the pipeline array
  # {
  #   '$replaceRoot': {
  #     'newRoot': '$users',
  #   },
  # },
  # sort user objects, like you want
  # {
  #   '$sort': {
  #     'name': 1,
  #   },
  # },
]

list(likes_collection.aggregate(pipeline))

In [212]:
df = pd.DataFrame.from_records(likes_collection.find({'is_liked': True}), columns=['happen_id', 'user_id', 'is_liked', 'is_disliked'])
df

,happen_id,user_id,is_liked,is_disliked
0,65fe4fc0aa58ed17295fa3bf,661f2d5e30ee4c696a1d15fd,True,False
1,66208de37b852a12bbe1b311,661f2d5e30ee4c696a1d15fd,True,False
2,66208de37b852a12bbe1b311,6621effacf09c02f9a9590d9,True,False


In [213]:
# https://stackoverflow.com/questions/55097226/mongodb-count-fi-true-in-aggregate-group
list(likes_collection.aggregate([
    {"$match": {"happen_id": "66208de37b852a12bbe1b311"}},
    {"$group": {"_id": "$name", "data": { "$first": "$$ROOT"},
                "liked": { "$sum": {"$cond": [{"$eq": ["$$ROOT.is_liked",True]},1,0]}},
                "disliked": {"$sum": {"$cond": [{"$eq": ["$$ROOT.is_disliked",True]},1,0]}}}},
    {"$project": {"liked_count": "$liked","disliked_count": "$disliked"}},
    {'$unset': ['_id']}]))

[{'liked_count': 2, 'disliked_count': 0}]

In [ ]:
# db.orders.aggregate([
#   {
#     $group: {
#       _id: {
#         country: '$data.nested.country',
#         product: '$data.product',
#       },
#       productCodes: {
#         $push: '$data.latest.sub.code',
#       },
#       uniqueCodes: {
#         $addToSet: '$data.latest.sub.code',
#       }
#     }
#   },
#   {
#     $group: {
#       _id: '$_id.country',
#       country: {
#         $first: '$_id.country',
#       },
#       products: {
#         $push: {
#           product: '$_id.product',
#           codes: {
#             $map: {
#               // run $filter+$size operations per each code
#               input: '$uniqueCodes',
#               as: 'code',
#               in: {
#                 code: '$$code',
#                 count: {
#                   $size: {
#                     $filter: {
#                       // collect same codes into one array
#                       // to be able to count them per product 
#                       input: '$productCodes',
#                       cond: {
#                         $eq: ['$$code', '$$this'],
#                       },
#                     },
#                   },
#                 },
#               },
#             },
#           },
#         },
#       },
#     },
#   },
#   // cleanup
#   {
#     $unset: ['_id'],
#   }
# ]);